## imports and librairies

In [ ]:
import pandas as pd
import lxml
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException

import time
from math import nan, isnan
from contextlib import suppress
from selenium.webdriver.firefox.options import Options







# URL and driver initialization

In [ ]:
url = 'https://www.bct.gov.tn/bct/siteprod/Tunibor_archive.jsp'
req = requests.get(url, headers={"Content-Type": "text"})
#path = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(executable_path=r"C:\Program Files (x86)\chromedriver.exe")
options = Options()
#options.headless = False
#driver = webdriver.Firefox(executable_path=r"C:\Program Files (x86)\geckodriver.exe")
driver.get("https://www.bct.gov.tn/bct/siteprod/Tunibor_archive.jsp")
driver.maximize_window()

## Creating the function that clicks on the calendar

In [ ]:
def click_btn(x):
    a=driver.find_element(By.CSS_SELECTOR,x)
    
    

    ActionChains(driver).click(a).perform()
    

## creating the function that clicks on the "OK" button after selecting the date

In [ ]:
def click_ok():
     WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '#output > div > div:nth-child(1) > form > p > span:nth-child(2) > input.button1'))).click()

## creating the function that moves to the next month 

In [ ]:
def click_next_month(x):
    a='#bct > div > table > thead > tr.headrow > td:nth-child(2) > div'
    click_cal()
    for i in range(x):
        if(check_exists_by_css_selector(a)):
            
            b=driver.find_element(By.CSS_SELECTOR,a)
            ActionChains(driver).click(b).perform()
            
        else:
            break

In [ ]:
def check_exists_by_css_selector(selector):
    try:
        driver.find_element(By.CSS_SELECTOR,selector)
    except NoSuchElementException:
        return False
    return True

## creating the function that verifies if the web element exists by its CSS selector 

In [ ]:
def check_exists(selector):
    try:
        driver.find_element(By.CSS_SELECTOR,selector)
    except ElementNotInteractableException:
            return False
    return True

In [ ]:
list_dates = []




tunibor = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="cours_archiv"]/div/div/div/center[2]/div/table'))).get_attribute(
    "outerHTML")


z=5

full_list=[]



In [ ]:
def scrape():
    dataframe=pd.DataFrame()
    for y in range(1,6):
        for x in range(1,8):
            try:
                click_cal()
                aa='/html/body/div/table/tbody/tr['+str(y)+']/td['+str(x)+']'
                l = list(range(1, 50))
                a='#bct > div> table > tbody > tr:nth-child('+str(y)+') > td:nth-child('+str(x)+')'
                if (check_exists_by_css_selector(a)):
                    z=click_btn(a)
                    dates=driver.find_element(By.XPATH,'//*[@id="f_date_c"]').get_attribute('value')
                    click_ok()
                    time.sleep(3)
                    table2 = WebDriverWait(driver, 3).until(EC.visibility_of_element_located(
                        (By.XPATH, '//*[@id="cours_archiv"]/div/div/div/center[2]/div/table'))).get_attribute("outerHTML")

                    df = pd.read_html(table2)
              
                    dataf = pd.concat(df, axis=0, ignore_index=True)
                    dataf = dataf.T
                    dataf['date'] = dates


                    dataframe = dataframe.append(dataf)

                    dataframe_transposed = dataframe.T
                
                    datacsv = dataframe.to_csv(r'C:\test\tunibor.csv', index=False,sep=';')
        
            except TimeoutException:
                continue
    click_next_month(1)
                

In [ ]:
def scrape2():
    dataframe=pd.DataFrame()
    for y in range(3,6):
        for x in range(5,8):
            try:
                driver.refresh()
                click_next_month(1)
                click_cal()
                aa='/html/body/div/table/tbody/tr['+str(y)+']/td['+str(x)+']'
                l = list(range(1, 50))
                a='#bct > div> table > tbody > tr:nth-child('+str(y)+') > td:nth-child('+str(x)+')'
                if (check_exists_by_css_selector(a)):
                    z=click_btn(a)
                    dates=driver.find_element(By.XPATH,'//*[@id="f_date_c"]').get_attribute('value')
                    click_ok()
                    time.sleep(3)
                    table2 = WebDriverWait(driver, 3).until(EC.visibility_of_element_located(
                        (By.XPATH, '//*[@id="cours_archiv"]/div/div/div/center[2]/div/table'))).get_attribute("outerHTML")

                    df = pd.read_html(table2)
              
                    dataf = pd.concat(df, axis=0, ignore_index=True)
                    dataf = dataf.T
                    dataf['date'] = dates


                    dataframe = dataframe.append(dataf)

                    dataframe_transposed = dataframe.T
                
                    datacsv = dataframe.to_csv(r'C:\test\tunibor.csv', index=False,sep=';')
        
            except TimeoutException:
                continue
    click_next_month(1)
                

## browsing a whole year 

In [ ]:


def browse(y):
    dataframe=pd.DataFrame()
    a=0
   
    while y<=5:
        for x in range(1,8):
            try:
                u='#bct >div>table >tbody>tr:nth-child({})> td:nth-child({})'.format(y,x)
           
                click_cal()
                #text=WebDriverWait(driver,10).until(
        #EC.visibility_of_element_located((By.CSS_SELECTOR,u))).text
            
                classs=WebDriverWait(driver,10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR,u))).get_attribute("class")
            
                #go=WebDriverWait(driver,10).until(
        #EC.presence_of_element_located((By.CSS_SELECTOR,'#bct > div.calendar > table > thead > tr.headrow > td:nth-child(4)'))).click()
                
           
            
            
                day = driver.find_element(By.CSS_SELECTOR, u)
                time.sleep(5)
                day.click()
                dates=driver.find_element(By.XPATH,'//*[@id="f_date_c"]').get_attribute('value')
           
                click_ok()
            
            
                if classs!="day":
                    continue
            
                response = requests.get(url)
                soup = BeautifulSoup(response.text, "html.parser")
            
                table = soup.find(id="cours_archiv")

            #response = requests.post('https://www.bct.gov.tn/bct/siteprod/Tunibor_archive.jsp')

                table2 = WebDriverWait(driver, 3).until(EC.visibility_of_element_located(
                    (By.XPATH, '//*[@id="cours_archiv"]/div/div/div/center[2]/div/table'))).get_attribute("outerHTML")

                df = pd.read_html(table2)
                
                dataf = pd.concat(df, axis=0, ignore_index=True)
                dataf = dataf.T
                dataf['date'] = dates


                dataframe = dataframe.append(dataf)

                dataframe_transposed = dataframe.T
                
                datacsv = dataframe.to_csv(r'C:\test\tunibor.csv', index=False,sep=';')
                print(dates)
                if y==5 and x==3:
                    break
            except (TimeoutException,ElementNotInteractableException):
                print("emptycell")
                continue
                
            
            
            
        y=y+1
        
        
                
            
    # full_list.append(df)
    
           
               
         
  
                
time.sleep(5)

In [ ]:

browse(1)


